In [1]:
import polars as pl

from sentence_transformers import SentenceTransformer

In [2]:
df = pl.read_parquet('data/processed/university_and_program.parquet')

In [3]:
df.head()

program,university_description,program_description,career_prospect
str,str,str,str
"""Bachelor of Science (Architect…","""We redefine design, education …","""The Undergraduate programme co…","""Architecture, Urban design, Ci…"
"""Bachelor of Engineering (Compu…","""We redefine design, education …","""Lead a successful professional…","""Cybersecurity specialist/analy…"
"""Bachelor of Science (Design an…","""We redefine design, education …","""Design Innovator, Innovation S…","""Applied machine learning engin…"
"""Bachelor of Engineering (Engin…","""We redefine design, education …","""EPD graduates are prepared for…","""Design engineer, Management co…"
"""Bachelor of Engineering (Engin…","""We redefine design, education …","""How do you decide what company…","""Aviation analyst, Corporate pl…"


In [4]:
columns_to_embed = ['university_description', 'program_description', 'career_prospect']
model_name = 'all-MiniLM-L6-v2'

In [5]:
model = SentenceTransformer(model_name)

for column in columns_to_embed:
    embedding_array = model.encode(df[column].to_list())
    embedding_column_names = {column + '_embeded_' + str(i): float for i  in range(embedding_array.shape[1])}
    df_embedded = pl.DataFrame(embedding_array, schema=embedding_column_names)
    df = pl.concat([df, df_embedded], how='horizontal')

In [6]:
df.head(10)

program,university_description,program_description,career_prospect,university_description_embeded_0,university_description_embeded_1,university_description_embeded_2,university_description_embeded_3,university_description_embeded_4,university_description_embeded_5,university_description_embeded_6,university_description_embeded_7,university_description_embeded_8,university_description_embeded_9,university_description_embeded_10,university_description_embeded_11,university_description_embeded_12,university_description_embeded_13,university_description_embeded_14,university_description_embeded_15,university_description_embeded_16,university_description_embeded_17,university_description_embeded_18,university_description_embeded_19,university_description_embeded_20,university_description_embeded_21,university_description_embeded_22,university_description_embeded_23,university_description_embeded_24,university_description_embeded_25,university_description_embeded_26,university_description_embeded_27,university_description_embeded_28,university_description_embeded_29,university_description_embeded_30,university_description_embeded_31,university_description_embeded_32,…,career_prospect_embeded_347,career_prospect_embeded_348,career_prospect_embeded_349,career_prospect_embeded_350,career_prospect_embeded_351,career_prospect_embeded_352,career_prospect_embeded_353,career_prospect_embeded_354,career_prospect_embeded_355,career_prospect_embeded_356,career_prospect_embeded_357,career_prospect_embeded_358,career_prospect_embeded_359,career_prospect_embeded_360,career_prospect_embeded_361,career_prospect_embeded_362,career_prospect_embeded_363,career_prospect_embeded_364,career_prospect_embeded_365,career_prospect_embeded_366,career_prospect_embeded_367,career_prospect_embeded_368,career_prospect_embeded_369,career_prospect_embeded_370,career_prospect_embeded_371,career_prospect_embeded_372,career_prospect_embeded_373,career_prospect_embeded_374,career_prospect_embeded_375,career_prospect_embeded_376,career_prospect_embeded_377,career_prospect_embeded_378,career_prospect_embeded_379,career_prospect_embeded_380,career_prospect_embeded_381,career_prospect_embeded_382,career_prospect_embeded_383
str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Bachelor of Science (Architect…","""We redefine design, education …","""The Undergraduate programme co…","""Architecture, Urban design, Ci…",0.008688,0.073831,-0.001575,-0.019872,0.072993,-0.020747,-0.047765,0.055445,0.039928,0.026293,-0.038317,0.061855,0.03594,0.058757,-0.019072,-0.007374,-0.070987,0.046221,-0.091606,-0.016127,-0.057109,-0.007607,0.04641,0.015605,-0.046362,-0.002131,-0.013758,-0.08543,-0.019007,-0.101654,0.020909,-0.005497,0.073397,…,-0.079741,-0.008681,0.069377,0.012784,-0.010443,0.017876,-0.036119,0.053104,0.019172,0.074196,-0.023345,0.004224,-0.092691,0.016044,-0.032889,-0.033977,0.030128,0.054265,-0.016162,-0.012371,0.006621,0.006457,-0.013931,-0.073886,0.037537,-0.060127,-0.017035,0.068026,0.054099,0.005551,-0.055582,0.029859,-0.057839,0.091465,0.040404,-0.01252,0.078221
"""Bachelor of Engineering (Compu…","""We redefine design, education …","""Lead a successful professional…","""Cybersecurity specialist/analy…",0.008688,0.073831,-0.001575,-0.019872,0.072993,-0.020747,-0.047765,0.055445,0.039928,0.026293,-0.038317,0.061855,0.03594,0.058757,-0.019072,-0.007374,-0.070987,0.046221,-0.091606,-0.016127,-0.057109,-0.007607,0.04641,0.015605,-0.046362,-0.002131,-0.013758,-0.08543,-0.019007,-0.101654,0.020909,-0.005497,0.073397,…,-0.071923,-0.026316,0.072887,-0.020698,-0.061888,0.048688,-0.016318,0.02967,0.028149,0.107787,-0.047017,-0.028653,0.059707,-0.094057,-0.023785,0.041886,0.046752,-0.069108,-0.033387,-0.031287,0.037114,0.022429,-0.063337

In [8]:
df.write_parquet('data/processed/embedded_university_and_progam.parquet')
df.write_parquet('app/data/embedded_university_and_progam.parquet')
df.write_csv('data/processed/embedded_university_and_progam.csv')